### 중앙치매센터 치매시설정보 크롤링
- https://www.nid.or.kr/info/facility_list.aspx

In [2]:
!pip show selenium

Name: selenium
Version: 4.12.0
Summary: 
Home-page: https://www.selenium.dev
Author: 
Author-email: 
License: Apache 2.0
Location: C:\Users\moony\anaconda3\Lib\site-packages
Requires: certifi, trio, trio-websocket, urllib3
Required-by: 


In [ ]:
! pip uninstall selenium

In [ ]:
! pip install selenium==3.14.1

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd
import pymongo as mg
import traceback

In [ ]:
client = mg.MongoClient(host='mongodb://localhost:27017')
database = client['facility_infos']
collection = database['dementia']

In [ ]:
# open chrome browser
browser = webdriver.Chrome(executable_path='../../../../chromedriver.exe')

In [ ]:
url = 'https://www.nid.or.kr/info/facility_list.aspx'
browser.get(url)
browser.implicitly_wait(3)

### Tags
['시설명', '주소', '시설분류정보', '전화번호', '홈페이지']<br>
['name', 'address', 'category', 'number', 'website']<br>
['시설명', '주소', '위도', '경도', '대분류', '중분류', '소분류', '전화번호', '홈페이지']<br>
['name', 'address', 'latitude', 'longitude', 'main_category', 'sub_category', 'sub_category_2', 'number', 'website']
- 상세 버튼 : #dev_tbody > tr:nth-child({1~10, 마지막페이지 8}) > td:nth-child(4) > a
- 상세 버튼 목록 : #dev_tbody > tr > td:nth-child(4) > a
- 닫기 버튼 : div > div.popTitle > a

- 페이지 버튼 목록 : #div_paging > div > ol > li > a
- 페이지 버튼 : #div_paging > div > ol > li:nth-child({1~10}, 마지막 1) > a
- 다음 페이지 목록 : xpath : //*[@id="div_paging"]/div/a/img[@alt="다음"]
- 시설명 : div.list_tbl_01.fix > table > tbody > tr:nth-child(1) > td:nth-child(2)
- 주소 : div.list_tbl_01.fix > table > tbody > tr:nth-child(2) > td
- 시설분류정보 : div.list_info > div.dx_fa_div > ul > li
- 전화번호 : div.list_tbl_01.fix > table > tbody > tr:nth-child(1) > td:nth-child(4)
- 홈페이지 : div.list_tbl_01.fix > table > tbody > tr:nth-child(3) > td


In [ ]:
facility_list = []
columns_name = ['name', 'address', 'category', 'number', 'website']

# 페이지 버튼 목록 가져오기
while True : 
    page_list = browser.find_elements_by_css_selector('#div_paging > div > ol > li > a')

    for index, page in enumerate(page_list) :
        detail_btns = browser.find_elements_by_css_selector('#dev_tbody > tr > td:nth-child(4) > a')
        for idx, btn in enumerate(detail_btns) :

            # 상세보기 클릭
            element = WebDriverWait(browser, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, '#dev_tbody > tr:nth-child({}) > td:nth-child(4) > a'.format(idx+1))))
            element.click()

            browser.switch_to.frame('frameLayer_frame') # 상세페이지로 browser 전환
            name = WebDriverWait(browser, 10).until(EC.presence_of_element_located((By.XPATH, '/html/body/div/div[2]/div/div[2]/div[2]/table/tbody/tr[1]/td[1]')))
            name = name.text
            address = browser.find_element_by_xpath('/html/body/div/div[2]/div/div[2]/div[2]/table/tbody/tr[2]/td').text
            category = browser.find_element_by_xpath('/html/body/div/div[2]/div/div[2]/div[1]/ul/li').text
            number = browser.find_element_by_xpath('/html/body/div/div[2]/div/div[2]/div[2]/table/tbody/tr[1]/td[2]').text
            website = browser.find_element_by_xpath('/html/body/div/div[2]/div/div[2]/div[2]/table/tbody/tr[3]/td').text
            facility_info = [name, address, category, number, website]
            facility_list.append(facility_info)

            # 상세페이지 닫기
            close_btn = browser.find_element_by_xpath('/html/body/div/div[1]/a/img[@alt="닫기"]')
            browser.execute_script("arguments[0].click();", close_btn)
            browser.switch_to.parent_frame()
        try: 
            next_page = browser.find_element_by_css_selector('div > ol > li:nth-child({})'.format(index+2))
            next_page.click()
        except:
            try:
                next_btn = browser.find_element_by_xpath('//*[@id="div_paging"]/div/a/img[@alt="다음"]')
                next_btn.click()
                pass
            except Exception as e:
                print(f"에러 발생: {str(e)}")                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           
                traceback.print_exc() 
                break
    break

In [ ]:
# DB에 추가
df_dementia_facility = pd.DataFrame(data=facility_list, columns=columns_name)
data_dict = df_dementia_facility.to_dict(orient='records')
collection.insert_many(data_dict)  

In [ ]:
browser.quit()

In [ ]:
cursor = collection.find({})
list_cursors = list(cursor)

In [ ]:
df_total = pd.DataFrame(list_cursors)

In [ ]:
df_total[:3]

In [ ]:
df_drop_id = df_total.drop('_id', axis=1)
df_drop_id[:3]

In [ ]:
df_drop_id.info()

In [ ]:
df_drop_dup = df_drop_id.drop_duplicates()
df_drop_dup.info()

In [ ]:
df_drop_dup.to_csv('./dementia_facility.csv')

In [ ]:
# {"content": {"$regex": "수술"}}
# {"hospital": {"$regex": "연세백퍼센트병원"}}
# {"name": {"$regex": "동동"}}ㅠ

In [ ]:
# MongoDB에 연결
client = mg.MongoClient("mongodb://localhost:27017")

# 데이터베이스와 컬렉션 선택
database = client['facility_infos']
collection = database['dementia_drop_duplicates']

# 자동 생성된 컬럼의 이름 확인
# 이 부분에서 자동 생성된 컬럼의 이름을 확인합니다.

# 컬럼 삭제 (예를 들어, "column_name"을 삭제하려면)
result = collection.update_many({}, {"$unset": {"_id": ""}})
result = collection.update_many({}, {"$unset": {"": ""}})

# 결과 확인
print(f"삭제된 문서 수: {result.modified_count}")